Soma total de vendas por dia, ordens com status de 'Delivered' e state = NY

In [0]:
%python
bronze_path   = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/bronze/'
silver_path   = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/silver/'
gold_path     = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/gold/'
resource_path = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/resource/origem/'
resource_path_volume = '/Volumes/infinitybigdata/azure/bikestore_resource/origem/'

In [0]:
%python
# criar varias tabelas temporárias de forma prática
silver_map = {
    #"tmp_silver_customers": f"{silver_path}/customers/",
    "tmp_silver_orders":    f"{silver_path}/orders/",
    #"tmp_silver_product":   f"{silver_path}/product/",
}
for view_name, path in silver_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))

In [0]:
%python
df_sales_ny_gold = spark.sql(
    """
    SELECT
    shipped_date,
    round(SUM(total_price),2) AS total_price,
    state,
    status
    FROM tmp_silver_orders
    WHERE state = 'NY'
    AND Status = 'DELIVERED'
    AND shipped_date IS NOT NULL
    GROUP BY shipped_date, state, status
    """
)

# salva em delta
df_sales_ny_gold.write\
    .mode("overwrite")\
    .format("delta")\
    .option("mergeSchema", "true")\
    .save(f"{gold_path}/sales_ny")